### Pandas trick #1: Keep rows having minimum values 

In [1]:
import pandas as pd

df = pd.DataFrame({
    "A": ["foo", "foo", "foo", "foo", "foo", "bar", "bar", "bar", "bar"],
    "B": ["one", "one", "one", "two", "two", "one", "one", "two", "two"],
    "C": ["small", "large", "large", "small", "small", "large", "small", "small", "large"],
    "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
    "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]
})
df

,A,B,C,D,E
0,foo,one,small,1,2
1,foo,one,large,2,4
2,foo,one,large,2,5
3,foo,two,small,3,5
4,foo,two,small,3,6
5,bar,one,large,4,6
6,bar,one,small,5,8
7,bar,two,small,6,9
8,bar,two,large,7,9


In [2]:
df.groupby("A")["E"].transform("min")

0    2
1    2
2    2
3    2
4    2
5    6
6    6
7    6
8    6
Name: E, dtype: int64

In [3]:
df.groupby("A")["E"].transform("min").eq(df["E"])

0     True
1    False
2    False
3    False
4    False
5     True
6    False
7    False
8    False
Name: E, dtype: bool

In [4]:
df.loc[df.groupby("A")["E"].transform("min").eq(df["E"])]

,A,B,C,D,E
0,foo,one,small,1,2
5,bar,one,large,4,6


### Pandas trick #2: named aggregation

In [ ]:
import pandas as pd
import numpy as np

animals = pd.DataFrame({
    "kind": ["cat", "dog", "cat", "dog"],
    "height": [9.1, 6.0, 9.5, 34.0],
    "weight": [7.9, 7.5, 9.9, 198.0]
})
animals

In [ ]:
animals.groupby("kind", as_index=False).agg(
    min_height=("height", "min"),
    max_height=("height", "max"),
    average_weight=("weight", np.mean)
)

In [ ]:
animals.groupby("kind", as_index=False).agg({"height": "min", "weight": "mean"})

### Pandas trick #3: where

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame(
    np.random.randn(6,4),
    index=list("abcdef"),
    columns=list("ABCD")
)
df

In [ ]:
df.where(df>0, -df)

In [ ]:
df.where(df>0, 0)

### Pandas trick #4: ngroup()

In [ ]:
import pandas as pd
df = pd.DataFrame({
    "a": [1,1,1,2,2,2],
    "b": [1,1,2,1,1,2]
})
df["idx"] = df.groupby(["a", "b"]).ngroup() + 1
df

### Pandas trick #5: resample & interpolate, pivot & stack

In [ ]:
import pandas as pd
df = pd.DataFrame({
    "Date": ["Q1", "Q2", "Q3", "Q4", "Q1", "Q2", "Q3", "Q4"],
    "Scenario": [1,1,1,1,1,1,1,1],
    "Path": [1,1,1,1,2,2,2,2],
    "rate_1": [0.02213, 0.02867, 0.03426, 0.01678, 0.02218, 0.02572, 0.01235, 0.03215],
    "rate_2": [0.02213, 0.02867, 0.03426, 0.01678, 0.02218, 0.02572, 0.01235, 0.03215]
})
df

In [ ]:
d = {
    "Q1": "2022-04-01",
    "Q2": "2022-07-01",
    "Q3": "2022-10-01",
    "Q4": "2023-01-01"
}
df["Date"] = df["Date"].map(d).fillna(df["Date"])
df

In [ ]:
df["Date"] = pd.to_datetime(df["Date"]).dt.to_period("M")
(
    df.set_index("Date")
      .groupby(["Scenario", "Path"])
      .resample("M")
      .ffill()
)

In [ ]:
(
    df.set_index("Date")
      .groupby(["Scenario", "Path"])
      .resample("M")
      .interpolate(method="linear")
      .drop(["Scenario", "Path"], axis=1)
)

In [ ]:
df = (
    df.set_index("Date")
      .groupby(["Scenario", "Path"])
      .resample("M")
      .asfreq()
      .drop(["Scenario", "Path"], axis=1)
      .interpolate(method="linear")
)
df

In [ ]:
cols = df.columns
df = (
    df.reset_index()
      .pivot(index=["Scenario", "Path"], columns="Date", values=cols)
      .stack(level=0)
      .reset_index()
)
df

### Pandas trick #6: mapping

In [ ]:
countries = pd.Series([
    'United States',
    'Canada',
    'Mexico',
    'Belgium',
    'United Kingdom',
    'Thailand'
])
groups = {
    'North America': ('United States', 'Canada', 'Mexico', 'Greenland'),
    'Europe': ('France', 'Germany', 'United Kingdom', 'Belgium')
}

In [ ]:
groups = {x: k for k, v in groups.items() for x in v}
groups

In [ ]:
countries.map(groups).fillna(countries)

In [ ]:
countries.map(groups).fillna(-999)

### Pandas trick #7: stack and unstack

In [6]:
import pandas as pd
import numpy as np

tuples = list(
    zip(
        *[
            ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
            ["one", "two", "one", "two", "one", "two", "one", "two"],
        ]
    )
)
index = pd.MultiIndex.from_tuples(tuples, names=["first", "second"])
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=["A", "B"])
df

A         B
first second                    
bar   one    -1.693951  0.944677
      two    -1.834788 -0.657229
baz   one    -0.588643 -0.188716
      two     0.025230  0.002564
foo   one    -0.383782 -0.172197
      two     0.076969 -1.020149
qux   one    -1.416278  1.320298
      two    -0.545749 -0.162633

In [7]:
stacked = df.stack()
stacked

first  second   
bar    one     A   -1.693951
               B    0.944677
       two     A   -1.834788
               B   -0.657229
baz    one     A   -0.588643
               B   -0.188716
       two     A    0.025230
               B    0.002564
foo    one     A   -0.383782
               B   -0.172197
       two     A    0.076969
               B   -1.020149
qux    one     A   -1.416278
               B    1.320298
       two     A   -0.545749
               B   -0.162633
dtype: float64

In [9]:
stacked.unstack()

A         B
first second                    
bar   one    -1.693951  0.944677
      two    -1.834788 -0.657229
baz   one    -0.588643 -0.188716
      two     0.025230  0.002564
foo   one    -0.383782 -0.172197
      two     0.076969 -1.020149
qux   one    -1.416278  1.320298
      two    -0.545749 -0.162633

### Pandas trick #8: multi-index, slice

In [10]:
import pandas as pd
import numpy as np

def mklbl(prefix, n):
    return ["%s%s" % (prefix, i) for i in range(n)]

miindex = pd.MultiIndex.from_product(
    [mklbl("A", 4), mklbl("B", 2), mklbl("C", 4), mklbl("D", 2)]
)

micolumns = pd.MultiIndex.from_tuples(
    [("a", "foo"), ("a", "bar"), ("b", "foo"), ("b", "bah")], names=["lvl0", "lvl1"]
)

dfmi = (
    pd.DataFrame(
        np.arange(len(miindex) * len(micolumns)).reshape(
            (len(miindex), len(micolumns))
        ),
        index=miindex,
        columns=micolumns,
    )
    .sort_index()
    .sort_index(axis=1)
)
dfmi

lvl0           a         b     
lvl1         bar  foo  bah  foo
A0 B0 C0 D0    1    0    3    2
         D1    5    4    7    6
      C1 D0    9    8   11   10
         D1   13   12   15   14
      C2 D0   17   16   19   18
...          ...  ...  ...  ...
A3 B1 C1 D1  237  236  239  238
      C2 D0  241  240  243  242
         D1  245  244  247  246
      C3 D0  249  248  251  250
         D1  253  252  255  254

[64 rows x 4 columns]

In [11]:
dfmi.loc[(slice("A1", "A3"), slice(None), slice("C1", "C3"))]

lvl0           a         b     
lvl1         bar  foo  bah  foo
A1 B0 C1 D0   73   72   75   74
         D1   77   76   79   78
      C2 D0   81   80   83   82
         D1   85   84   87   86
      C3 D0   89   88   91   90
         D1   93   92   95   94
   B1 C1 D0  105  104  107  106
         D1  109  108  111  110
      C2 D0  113  112  115  114
         D1  117  116  119  118
      C3 D0  121  120  123  122
         D1  125  124  127  126
A2 B0 C1 D0  137  136  139  138
         D1  141  140  143  142
      C2 D0  145  144  147  146
         D1  149  148  151  150
      C3 D0  153  152  155  154
         D1  157  156  159  158
   B1 C1 D0  169  168  171  170
         D1  173  172  175  174
      C2 D0  177  176  179  178
         D1  181  180  183  182
      C3 D0  185  184  187  186
         D1  189  188  191  190
A3 B0 C1 D0  201  200  203  202
         D1  205  204  207  206
      C2 D0  209  208  211  210
         D1  213  212  215  214
      C3 D0  217  216  219  218
         D1  221  220  223  222
   B1 C1 D0  233  232  235  234
         D1  237  236  239  238
      C2 D0  241  240  243  242
         D1  245  244  247  246
      C3 D0  249  248  251  250
         D1  253  252  255  254

In [12]:
dfmi.loc[("A0", "B0", slice(None), "D1")]

lvl0          a       b    
lvl1        bar foo bah foo
A0 B0 C0 D1   5   4   7   6
      C1 D1  13  12  15  14
      C2 D1  21  20  23  22
      C3 D1  29  28  31  30